In [ ]:
import torch
print(torch.cuda.is_available())

### **Preproccessing the Data**

In [ ]:
import pandas as pd
import torch
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import transforms
from sklearn.model_selection import train_test_split
import logging
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image = self.data[idx]
        label = self.labels[idx]
        return image, label

In [ ]:
def split_into_train_validation_test(df: pd.DataFrame, random_state: int, test_size: float):
    # X - pixels of images
    pixels = [col for col in df.columns if col.startswith('pixel')]
    X = df[pixels]
    y = np.array(df['label'])
    
    # Split into train & test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state=random_state)
    # Split into train & validation
    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = test_size, random_state=random_state)

    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
def data_preparation(data: pd.DataFrame, labels, device='cuda'):
    # Reshape the data to match the image dimensions (3, 32, 32)
    data = data.values.reshape(-1, 3, 32, 32)

    # Convert data to float32 and normalize to the range [0, 1]
    data = data.astype(np.float32) / 255.0

    # Convert NumPy arrays to PyTorch tensors
    data_tensors = torch.tensor(data).to(device)
    label_tensors = torch.tensor(labels).to(device)

    # Normlization by the mean of each channel
    # Standardize
    normalize_transform = transforms.Normalize(mean=[0.0, 0.0, 0.0], std=[1.0, 1.0, 1.0])
    data_tensors = normalize_transform(data_tensors)

    # Create a CustomDataset instance
    dataset = CustomDataset(data_tensors, label_tensors)

    # Create a DataLoader
    batch_size = 128
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return loader

In [ ]:
df_imbalance = pd.read_csv("/kaggle/input/cifar-10-and-3-classes-from-cifar-100-imbalance/dataset.csv")

X_train_imbalance, X_val_imbalance, X_test_imbalance, y_train_imbalance, y_val_imbalance, y_test_imbalance = split_into_train_validation_test(df_imbalance, random_state = 42, test_size = 0.2)
train_loader_imbalance = data_preparation(X_train_imbalance, y_train_imbalance)
val_loader_imbalance = data_preparation(X_val_imbalance, y_val_imbalance)
test_loader_imbalance = data_preparation(X_test_imbalance, y_test_imbalance)

In [ ]:
df = pd.read_csv("/kaggle/input/perfect-dataset/perfect_dataset.csv")

X_train, X_val, X_test, y_train, y_val, y_test = split_into_train_validation_test(df, random_state = 66, test_size = 0.2)
train_loader = data_preparation(X_train, y_train)
val_loader = data_preparation(X_val, y_val)
test_loader = data_preparation(X_test, y_test)

## Resnet18 model - Tensorboard

In [ ]:
import torchvision.models as models
import torch.nn as nn
import os
from tensorboardX import SummaryWriter

In [ ]:
def evaluate_model(model, dataloader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():
        for images, labels in dataloader:
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy

In [ ]:
def resnet18_model_with_tensorboard(num_epochs: int, train_loader, val_loader, num_classes: int, device='cuda'):
    # Define the ResNet18 model
    model = models.resnet18(pretrained=False).to(device)
    model.fc = nn.Linear(512, num_classes).to(device)
    
    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-5)

    # Set up TensorBoard writer
    writer = SummaryWriter('logs')
    os.makedirs('/kaggle/working/MyResNet18Models')

    num_train_batches = len(train_loader)
    for epoch in range(num_epochs):
        model.train()  # Set the model to training mode
        train_loss = 0.0

        for i, (images, labels) in enumerate(train_loader):
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward and optimize
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Calculate average training loss for the epoch
        train_loss /= num_train_batches
        
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        num_val_batches = len(val_loader)

        with torch.no_grad():  
            for batch_idx, (val_inputs, val_labels) in enumerate(val_loader):
                val_outputs = model(val_inputs)
                val_loss_batch = criterion(val_outputs, val_labels)
                val_loss += val_loss_batch.item()

        val_loss /= num_val_batches
        
        accuracy_train = evaluate_model(model, train_loader)
        accuracy_val = evaluate_model(model, val_loader)

        # Write to TensorBoard
        writer.add_scalar('Loss/train', train_loss, epoch)
        writer.add_scalar('Loss/Val', val_loss, epoch)
        writer.add_scalar('Accuracy/train', accuracy_train , epoch)
        writer.add_scalar('Accuracy/Val', accuracy_val , epoch)
        
        torch.save(model.state_dict(), f'/kaggle/working/MyResNet18Models/resnet18_model_{epoch}.pth') # save each model
        
    # Close the TensorBoard writer
    writer.close()
    
    return model

In [ ]:
model_imbalance = resnet18_model_with_tensorboard(200, train_loader_imbalance, val_loader_imbalance, 13)

In [ ]:
model = resnet18_model_with_tensorboard(200, train_loader, val_loader, 13)

In [ ]:
accuracy_imbalance = evaluate_model(model_imbalance, val_loader_imbalance)
print("Test Accuracy: {:.2f}%".format(accuracy_imbalance))

In [ ]:
accuracy = evaluate_model(model, val_loader)
print("Test Accuracy: {:.2f}%".format(accuracy))

In [ ]:
def load_model(model, model_path):
    model = models.resnet18(pretrained=False)
    model.fc = nn.Linear(512, 13)
    model.load_state_dict(torch.load(model_path))
    return model

In [1]:
%load_ext tensorboard

In [4]:
%tensorboard --logdir logs --host localhost --port 6007

In [5]:
%tensorboard --logdir logs --host localhost --port 6008

In [6]:
%tensorboard --logdir logs --host localhost --port 6009

In [8]:
%tensorboard --logdir logs --host localhost --port 6011